# Mettler Toledo WXS205SDU

| Summary | Image |
|------------|--------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.mt.com/gb/en/home/products/Industrial_Weighing_Solutions/high-precision-weigh-sensors/weigh-module-wxs205sdu-15-11121008.html" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial / RS-232</li> <li><b>Communication Level:</b> Firmware (documentation shared by OEM)</li> <li>VID:PID <code>????:????</code></li> <li>High-precision fine balance with various adapters available.</li> </ul> | <div style="width:320px; text-align:center;"> ![shaker](img/mettler_toledo_wx_scale.png) <br><i>Figure: Mettler Toledo WXS205SDU used for gravimetric liquid transfer verification</i> </div> |

---
## Setup (Physical)

The scale consists of multiple untis":
- load cell:
- electronic unit: 
- display (optional)

The scale comes with an RS-232 serial port.
You'll probably want to use a USB to serial adapter to connect it to your computer.
Any $10 generic USB to serial adapter should work (e.g. something that uses FTDI).


---
## Setup (Programmatic)

In [ ]:
from pylabrobot.scales import Scale
from pylabrobot.scales.mettler_toledo_backend import MettlerToledoWXS205SDUBackend

In [ ]:
backend = MettlerToledoWXS205SDUBackend(port="/dev/cu.usbserial-110")
scale = Scale(name="scale", backend=backend, size_x=0, size_y=0, size_z=0)

await scale.setup()


0.00148

```{Warning}
This scale has a **'warm-up'** time after plugging in that is documented as 60-90 minutes by Mettler Toledo depending on the document you look at.
In our experience, 30 minutes is sufficient.
If you try to take a measurement before this time, you will likely get a "Command understood but currently not executable (balance is currently executing another command)" error.
Sometimes plugging the power cord in and out will make things work faster.
```

Note: This scale is used by the Hamilton Liquid Verification Kit (LVK).


---
## Usage

Scales are very simple machines with only 3 features:

- {meth}`~pylabrobot.scales.scale.Scale.get_weight`: Read the weight from the scale in grams.
- {meth}`~pylabrobot.scales.scale.Scale.tare`: Tare the scale.
- {meth}`~pylabrobot.scales.scale.Scale.zero`: Zero the scale.


In [ ]:
await scale.get_weight(timeout="stable")

This scale provides various timeouts:

- `"stable"`: The time to wait for the scale to stabilize before reading the weight. Note that this may take a very long time to finish if the scale cannot take a stable reading. If you're not using the air enclosure, even being near the scale can cause it to never stabilize.
- 0: Read the value immediately
- n>0: Try to get a stable value for n seconds. If the value is stable before n seconds, return it immediately. Otherwise, return the value after n seconds.

These parameters are available for {meth}`~pylabrobot.scales.mettler_toledo_backend.MettlerToledoWXS205SDUBackend.get_weight`, {meth}`~pylabrobot.scales.mettler_toledo_backend.MettlerToledoWXS205SDUBackend.tare`, and {meth}`~pylabrobot.scales.mettler_toledo_backend.MettlerToledoWXS205SDUBackend.zero`.

In [4]:
await scale.get_weight(timeout=0)

0.00148

### Example: getting timing information

Let's say you wanted to determine how long it takes to take a measurement. In PyLabRobot, this is easy:

In [6]:
import time
import numpy as np

l = []
for i in range(10):
  t0 = time.monotonic_ns()
  await scale.get_weight(timeout="stable")
  t1 = time.monotonic_ns()
  l.append((t1 - t0) / 1e6)

print(f"{np.mean(l):.2f} ms ± {np.std(l):.2f} ms")

100.44 ms ± 6.78 ms
